In [ ]:
import warnings
warnings.filterwarnings("ignore")

import os
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from numpy import mean, std
from sklearn.model_selection import KFold
import operator
import pickle
from joblib import dump, load

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score,precision_score,roc_auc_score,recall_score
from sklearn.model_selection import GridSearchCV, train_test_split, cross_val_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer  
from sklearn.feature_selection import SelectFromModel
from sklearn.pipeline import Pipeline

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, RandomForestClassifier, ExtraTreesClassifier, VotingClassifier


# # import nltk
# # from nltk.corpus import stopwords
# # from nltk.stem import PorterStemmer
# # from textblob import TextBlob, Word
# nltk.download('stopwords')
# nltk.download('punkt')

# stopwords = stopwords.words('english')
# # from nltk.stem import WordNetLemmatizer

from db_wrapper import db

In [ ]:
import sys
sys.executable

'c:\\program files\\python35\\python.exe'

In [ ]:
tfidf_vect = load(r"G:\Reclassification_Algorithm\feature.joblib") # Loading model joblib
model=load(r"G:\Reclassification_Algorithm\model.joblib") # Loading tfidf_vectorizer joblib
vocab_dict=load(r"G:\Reclassification_Algorithm\vocab.joblib")

In [2]:
select_query = "Select * from xx.zz"

stat,result_set = db.select(select_query)
test_data = pd.DataFrame(result_set)

In [ ]:
test_data.rename({"lifnr":"vendor id"}, axis=1, inplace=True)

In [ ]:
test_data["vendor id"].unique()

array(['0000381111', '0000000276', '0000000525', ..., '0008349661',
       '0000320931', '0003008976'], dtype=object)

In [ ]:
test_data.package.unique()

array(['Sales','Supply Chain Overhead', 'Taxes',
       'Variable compensation'], dtype=object)

In [1]:
select_query = "Select * from [xxx].[yyyyy]"

stat,result_set = db.select(select_query)
vendor_data = pd.DataFrame(result_set)

In [ ]:
vendor_data.columns

Index(['(INV) CAPEX/OPEX', '(INV) Document Number (InvoiceId)',
       '(INV)Company Country (Company Country)',
       '(INV)Company Zone (Company Zone)',
       '(INV)Contract Id + Line item (Contract Id + Line item)',
      dtype='object')

In [ ]:
def add0(x):
    try:
        if(len(x)!=10):
            padding_size=(10-len(x))
            x=x.rjust(10, "0")
            
        else:
            x
    except:
        print("error in ",x)
        
    return(float(x))
        

In [ ]:
test_data["vendor id"]=test_data["vendor id"].astype(str)
test_data["vendor id"]=test_data["vendor id"].apply(lambda x: add0(x))

In [ ]:
test_data["vendor id"]=test_data["vendor id"].round(decimals=0).astype(object)

In [ ]:
vendor_data2["vendor id"]=vendor_data2["vendor id"].astype(str)
vendor_data2["vendor id"]=vendor_data2["vendor id"].apply(lambda x: add0(x))
vendor_data2["vendor id"]=vendor_data2["vendor id"].round(decimals=0).astype(object)

In [ ]:
test_data["vendor id"][:3]


0    381111
1       276
2       525
Name: vendor id, dtype: object

In [ ]:
vendor_data2["vendor id"][:3]

0    346426
1    383062
2    382548
Name: vendor id, dtype: object

In [ ]:
def punc_reomval(x):
    
    punctuations = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
    no_punct = ""
    for char in x:
        if char not in punctuations:
            no_punct = no_punct + char
    return no_punct

In [ ]:
def preprocessing(x):
  
    x = re.sub(r'\W', ' ', str(x))
    x = re.sub(r'\s+[a-zA-Z]\s+', ' ', x)
    x = re.sub(r'\^[a-zA-Z]\s+', ' ', x) 
    x = re.sub(r'\s+', ' ', x, flags=re.I)
    return x

In [ ]:
def appostrophe_replace(x):
    return(x.replace("'", " ").replace('"', " "))

In [ ]:
test_data["purchase_order_description"]=test_data["purchase_order_description"].apply(lambda x: punc_reomval(x))
test_data["purchase_order_description"]=test_data["purchase_order_description"].apply(lambda x: preprocessing(x))
test_data["purchase_order_description"]=test_data["purchase_order_description"].apply(lambda x: appostrophe_replace(x))

In [ ]:
# test_data[test_data["purchase_order_description"].str.contains("MXP")]

In [ ]:
import ven_map
from ven_map import O2O, result

In [ ]:
# data=pd.read_excel(r'G:\Users\C888653\Reclassification_Algorithm\ghq_data_with Vendor id.xlsx')

In [ ]:
# test_data[test_data["package"]=="Technology"]["vendor id"].unique()

In [ ]:
# vendor_data2["vendor id"].unique()

In [ ]:
# tech=test_data[test_data.package=="Technology"]

In [ ]:
# tech=test_data[test_data.package=="Technology"]

In [ ]:
# tech["vendor id"]=tech["vendor id"].astype(str)
# tech["vendor id"]=tech["vendor id"].apply(lambda x: add0(x))
# tech["vendor id"]=tech["vendor id"].round(decimals=0).astype(object)

In [ ]:
# tech["vendor id"].unique()

In [ ]:
oTo_o=O2O(test_data)

In [ ]:
oTo_o.count_pack_venid()[:3]

vendor id
1.0    1
Name: package, dtype: int64

In [ ]:
oTo_o.count_subp_venid()[:3]

vendor id
1.0    1
Name: subpackage, dtype: int64

In [ ]:
oTo_o.uniq_p

array(['Sales', 'Supply Chain Overhead', 'Taxes',
       'Variable compensation'], dtype=object)

In [ ]:
oTo_o.uniq_subp

array(['Commercial Agreement - Distributors', None, 'Other',
       'International Mobility', 'Temps Non-Payroll',
       'Lease of Real Estate', 'Cleaning and Gardening',
      dtype=object)

In [ ]:
len(oTo_o.uniq_venid)

1901

In [ ]:
r=result(vendor_data2,tech)
df=r.result_O2O() 

In [ ]:
df["reassign_pack"].value_counts()

Unassigned    721
Technology    301
Name: reassign_pack, dtype: int64

In [ ]:
df["reassign_subpack"].value_counts()

Unassigned              837
Applications Support     63
Application Licenses     35
Security                 34
Workplace                34
Telecom Services         19
Name: reassign_subpack, dtype: int64

In [ ]:
df.shape

(1035, 18)

In [ ]:
o2o=df[~((df["reassign_pack"]=="Unassigned" )  | (df["reassign_subpack"]=="Unassigned" ))]

In [ ]:
o2o.shape

(185, 18)

In [ ]:
o2o["reassign_pack"].value_counts()

Technology    185
Name: reassign_pack, dtype: int64

In [ ]:
o2m=df[((df["reassign_subpack"]=="Unassigned" ) | (df["reassign_pack"]=="Unassigned" ) )]

In [ ]:
o2m.shape[0]+o2o.shape[0]

1035

In [ ]:
ip_data = pd.read_excel(r'G:\Users\Reclassification_Algorithm\Trainning dataset.xlsx')

In [ ]:
testData=o2m[~o2m["po_key"].isin(ip_data["po_key"])]

In [ ]:
testData.shape

(850, 18)

In [ ]:
testData.isna().sum()

company_code                                0
description                                29
gl_code                                     0
gl_code_desc                                0
gpo categories                             29
vendor id                                   0
material_group_description                  0
package                                     0
po_key                                      0
po_total_amount_ordered_in_usd_currency     2
purchase_order_description                  0
sakto                                       0
subpackage                                  0
vendor_name                                 0
p_Match                                     0
sub_p_Match                                 0
reassign_pack                               0
reassign_subpack                            0
dtype: int64

In [ ]:
# string="MXP Aug'19"
# string.replace("'", ' ')

In [ ]:
from scipy.sparse import csr_matrix, hstack
def add_feature(X, feature_to_add):
    return hstack([X, csr_matrix(feature_to_add).T], 'csr')

In [ ]:
# testData["purchase_order_description"]=testData["purchase_order_description"].apply(lambda x: punc_reomval(x))
# testData["purchase_order_description"]=testData["purchase_order_description"].apply(lambda x: preprocessing(x))
# testData["purchase_order_description"]=testData["purchase_order_description"].apply(lambda x: appostrophe_replace(x))

In [ ]:
un_tf_idf_vect=tfidf_vect.transform(testData["purchase_order_description"].values)

In [ ]:
un_tf_idf_vect

<850x3150 sparse matrix of type '<class 'numpy.float64'>'
	with 4303 stored elements in Compressed Sparse Row format>

In [ ]:
test_vectorized = add_feature(un_tf_idf_vect, pd.Series(testData["purchase_order_description"].values).str.len())

In [ ]:
test_vectorized

<850x3151 sparse matrix of type '<class 'numpy.float64'>'
	with 5153 stored elements in Compressed Sparse Row format>

In [ ]:
test_df=pd.DataFrame(test_vectorized.todense(), index=testData["purchase_order_description"].values, columns=list(vocab_dict.values()))

In [ ]:
test_df.shape

(850, 3151)

In [ ]:
predicted=np.array(model.predict(test_df))

In [ ]:
test_df["predicted"]=predicted

In [ ]:
test_df["predicted"]

SSF Code Review                                Digital Security
CASB - NAZ                                     Digital Security
Pentest - People MDM                        Application Support
Gender Equality Comms                       Application Support
PO Extension of GCC Tickets/SRs for GOP     Application Support
SOW-MOUTS-2019-008 PMO Service              Application Support
CMS Support Azure h2 2019                            Datacenter
Azure Microsoft Q1                                   Datacenter
2019 IT2 Subscription and Maintenance       Application Support
Pentest - OPR Meeting                       Application Support
intralinks 2019 Q1                                   Datacenter
Vtex SaaS Platform Payment                  Application Support
IT2 project IaaS DB support GHQ                      Datacenter
GLS Red                                     Application Support
IT2 project IaaS DB support GHQ                      Datacenter
Vtex SaaS Platform Payment              

In [ ]:
testData.drop(['p_Match', 'sub_p_Match', 'reassign_pack', 'reassign_subpack'], axis=1, inplace=True)

In [ ]:
testData["predicted"]=predicted

In [ ]:
testData["predicted"].value_counts()

Application Support     579
Workplace                90
Digital Security         76
Datacenter               74
Application Licenses     16
Telecom Services         13
Mobile Services           2
Name: predicted, dtype: int64

[po_key, po_decription, predicted, vendor logic flag]

In [ ]:
o2o.drop(['p_Match', 'sub_p_Match', 'reassign_pack'], axis=1, inplace=True)

In [ ]:
o2o.rename({"reassign_subpack": "predicted"}, axis=1, inplace=True)

In [ ]:
o2o["flag"]=np.ones(o2o.shape[0])

In [ ]:
testData["flag"]=np.zeros(testData.shape[0])

In [ ]:
final_res1=o2o[[ 'po_key', 
       'purchase_order_description', 
       'predicted', 'flag']]

In [ ]:
final_res2=testData[[ 'po_key', 
       'purchase_order_description', 
       'predicted', 'flag']]

In [ ]:
final_res=pd.concat([final_res1, final_res2], axis=0)

In [ ]:
final_res.reset_index(drop=True, inplace=True)

In [ ]:
final_res.shape

(1035, 4)

In [ ]:


final_res["purchase_order_description"]=final_res["purchase_order_description"].apply(lambda x: punc_reomval(x))
final_res["purchase_order_description"]=final_res["purchase_order_description"].apply(lambda x: preprocessing(x))
final_res["purchase_order_description"]=final_res["purchase_order_description"].apply(lambda x: appostrophe_replace(x))